In [1]:
!pip install tpot

     |████████████████████████████████| 92kB 3.9MB/s 
     |████████████████████████████████| 163kB 16.9MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp36-none-any.whl size=11956 sha256=9c368fd1e2e146fe79ee5301c3a98d7a8bfe52628ab0e216bf014affa73da54c
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit


# The purpose of this notebook is to use autoML to uncover some classifers and establish a baseline

https://www.datacamp.com/community/tutorials/tpot-machine-learning-python where I learned how to do this

## Import Necessary Libraries

In [4]:
import pandas as pd
import os
import glob
import shutil
from pandas import ExcelWriter


import numpy as np
import pandas as pd
import seaborn.apionly as sns
import matplotlib.pyplot as plt


import numpy as np
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier
from tpot import TPOTRegressor

In [3]:
from google.colab import files
uploaded = files.upload()

Saving PiotroskyTraining_ClassifierReady.xlsx to PiotroskyTraining_ClassifierReady.xlsx


In [9]:
# DotCom1 = pd.read_excel('/content/fin_Performance2001_Learning_Values_OneYearFundamentals.xlsx', header=0, parse_dates=[0])
features = pd.read_excel('/content/PiotroskyTraining_ClassifierReady.xlsx', header=0, parse_dates=[0])


In [10]:
features.head()

,Unnamed: 0,Quarter Date,Security,Name,Sector,ROA,CFO,ROA_Change,Accrual,LT_Debt_To_Asset_change,Current_Ratio_Change,No_Shares_Change,Gross_Margin_Change,Asset_Turnover_Change,Fscore,BOOK_VAL_PER_SH,ClosingPrice,BM,StartingPrice,OneYearPrice,TwoYearPrice,OneYearReturn,TwoYearReturn,Outperform,Underperform
0,0,2007-12-31,0111145D UN Equity,Nicor Inc,Utilities,1,1,1,1,0,1,0,1,0,6,20.139780,42.35,0.475556,42.32,34.96,40.66,-0.173913,-0.039225,1,0
1,2,2007-12-31,0167866D UN Equity,Petrohawk Energy Corp,Energy,1,1,0,1,1,0,0,0,1,5,11.743936,17.31,0.678448,15.97,19.73,23.58,0.235441,0.476518,1,0
2,3,2007-12-31,0202445Q UN Equity,EMD Millipore Corp,"Consumer, Non-cyclical",1,1,0,1,1,1,0,0,0,5,19.866647,73.18,0.271476,66.08,55.43,70.03,-0.161168,0.059776,1,0
3,4,2007-12-31,0203524D UN Equity,National Semiconductor Corp,Technology,1,1,0,1,1,1,1,1,0,7,1.071070,22.64,0.047309,19.02,10.26,13.68,-0.460568,-0.280757,0,1
4,6,2007-12-31,0226226D UN Equity,Centex LLC,"Consumer, Cyclical",0,1,0,1,1,1,0,0,1,5,34.261831,25.26,1.356367,26.67,8.32,NaN,-0.688039,NaN,0,1


In [11]:
features =features.iloc[np.random.permutation(len(features))]
features=features.reset_index(drop=True)

In [12]:
# features['class']=features['Performance'].map({'Best':1, 'Bad':2})
features = features.drop(columns=['Unnamed: 0', 'Quarter Date','Security', 'Name', 'Sector', 'ClosingPrice', 'StartingPrice', 'OneYearPrice', 'TwoYearPrice', 'OneYearReturn', 'TwoYearReturn', 'Underperform'])


In [13]:
features_class  = features['Outperform'].values

In [14]:
features

,ROA,CFO,ROA_Change,Accrual,LT_Debt_To_Asset_change,Current_Ratio_Change,No_Shares_Change,Gross_Margin_Change,Asset_Turnover_Change,Fscore,BOOK_VAL_PER_SH,BM,Outperform
0,1,1,1,1,0,0,0,1,0,5,19.571048,0.636457,0
1,1,1,0,1,0,1,1,0,0,5,9.294643,1.019149,0
2,1,1,1,1,0,0,0,1,0,5,14.519646,0.832800,1
3,1,1,1,1,0,1,0,0,1,6,20.943991,0.625007,1
4,1,1,1,1,0,0,1,0,0,5,22.137525,1.066322,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23510,0,1,1,1,0,0,0,1,0,4,7.086807,2.567684,0
23511,0,0,1,0,1,1,0,0,1,4,7.717263,0.301809,0
23512,1,1,0,1,0,1,0,0,0,4,19.729729,0.856325,0
23513,0,1,0,1,1,0,1,0,1,5,9.421074,0.630594,0


In [15]:

# split into training and test set
training_indices, validation_indices = training_indices, testing_indices = train_test_split(features.index,
                                                                                            stratify = features_class,
                                                                                            train_size=0.75, test_size=0.25)

training_indices.size, validation_indices.size

(17636, 5879)

# Use AutomML to uncover some useful models for classifying
## Limit the runtime to 2 minutes

In [18]:
tpot = TPOTClassifier(verbosity=2, max_time_mins=2, max_eval_time_mins=0.04, population_size=15)

tpot.fit(features.drop('Outperform',axis=1).loc[training_indices].values,
         features.loc[training_indices,'Outperform'].values)


Generation 1 - Current best internal CV score: 0.5248925040198482
Generation 2 - Current best internal CV score: 0.5248925040198482
Generation 3 - Current best internal CV score: 0.5526192662113518
2.02 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: KNeighborsClassifier(input_matrix, n_neighbors=36, p=1, weights=distance)


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=100,
               log_file=<ipykernel.iostream.OutStream object at 0x7f7707fe59e8>,
               max_eval_time_mins=0.04, max_time_mins=2, memory=None,
               mutation_rate=0.9, n_jobs=1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=15,
               random_state=None, scoring=None, subsample=1.0, template=None,
               use_dask=False, verbosity=2, warm_start=False)

## Test results, odd that it performed so well?

In [19]:
tpot.score(features.drop('Outperform',axis=1).loc[training_indices].values,
         features.loc[training_indices,'Outperform'].values)

1.0

# Test Against Test Data of 2018

In [20]:
from google.colab import files
uploaded = files.upload()

Saving PiotroskyTest_ClassifierReady.xlsx to PiotroskyTest_ClassifierReady.xlsx


In [30]:
Russ2018 = pd.read_excel('/content/PiotroskyTest_ClassifierReady.xlsx', header=0, parse_dates=[0])

In [31]:
Russ2018 =Russ2018.iloc[np.random.permutation(len(Russ2018))]
Russ2018=Russ2018.reset_index(drop=True)

In [32]:
Russ2018.head()

,Unnamed: 0,Quarter Date,Security,Name,Sector,ROA,CFO,ROA_Change,Accrual,LT_Debt_To_Asset_change,Current_Ratio_Change,No_Shares_Change,Gross_Margin_Change,Asset_Turnover_Change,Fscore,BOOK_VAL_PER_SH,ClosingPrice,BM,StartingPrice,OneYearPrice,TwoYearPrice,OneYearReturn,TwoYearReturn,Outperform,Underperform
0,2264,2018-12-31,RGLD UW Equity,Royal Gold Inc,Basic Materials,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,5.0,32.161852,85.65,0.375503,86.87,112.39,139.93,0.293772,0.610798,1,0
1,1193,2018-03-31,GPN UN Equity,Global Payments Inc,"Consumer, Non-cyclical",1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,23.837916,NaN,NaN,113.33,144.08,161.64,0.271332,0.426277,1,0
2,1411,2018-06-30,IOSP UW Equity,Innospec Inc,Basic Materials,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,4.0,33.782770,76.55,0.441316,78.75,92.48,76.80,0.174349,-0.024762,1,0
3,2314,2018-06-30,RRR UW Equity,Red Rock Resorts Inc,"Consumer, Cyclical",1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,6.0,3.666730,33.50,0.109455,34.07,19.83,11.69,-0.417963,-0.656883,0,1
4,1769,2018-12-31,MPWR UW Equity,Monolithic Power Systems Inc,Technology,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,6.0,14.387946,116.25,0.123767,129.54,175.21,265.01,0.352555,1.045777,1,0


In [33]:
Russ2018 = Russ2018.drop(columns=['Unnamed: 0', 'Quarter Date','Security', 'Name', 'Sector', 'ClosingPrice', 'StartingPrice', 'OneYearPrice', 'TwoYearPrice', 'OneYearReturn', 'TwoYearReturn', 'Underperform'])

Russ2018_class  = Russ2018['Outperform'].values



training_indices, validation_indices = training_indices, testing_indices = train_test_split(Russ2018.index,
                                                                                            stratify = Russ2018_class,
                                                                                            train_size=0.01, test_size=0.99)

training_indices.size, validation_indices.size

(82, 8175)

In [34]:
Findout =Russ2018.iloc[np.random.permutation(len(Russ2018))]
Findout=Findout.reset_index(drop=True)

# Just over guessing, not bad for a baseline!

In [36]:
tpot.score(Russ2018.drop('Outperform',axis=1).loc[validation_indices].values,
           Russ2018.loc[validation_indices, 'Outperform'].values)

Imputing missing values in feature set


0.5042201834862385